### Import required libraries

In [ ]:
import httpx
import json
import math
import os

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from tqdm import tqdm

### Setting mapbox access token

You can easily get your own mapbox access token via the following link: https://account.mapbox.com/access-tokens/

In [ ]:
mapbox_access_token = 'pk.eyJ1Ijoia2Npbm5heSIsImEiOiJja3lkMDJhcmUwODJmMm9zMjhob2V1N296In0.ymuiGnyaAIJSFC71UwlZYQ'
px.set_mapbox_access_token(mapbox_access_token)

### Configuration 
**load_from_file**  
set to True after having run the notebook once to prevent re-scraping the data when already available

In [ ]:
load_from_file = True

### Helper functions

In [ ]:
def combine_csvs_in_dir_to_df(dir_name):
    df = pd.DataFrame()
    for root, directories, files in os.walk(dir_name):
        for file in tqdm(files):
            df = df.append(pd.read_csv(f'{dir_name}/{file}'))
    return df

### Load in datasets

In [ ]:
df_postcodes = pd.read_csv('data/2019_postcodes.csv', low_memory=False)
df_postcodes = df_postcodes[df_postcodes['rgn'] == 'E12000007']

LSOAS_all_of_london = list(df_postcodes['lsoa11'].unique())

df_income = pd.read_csv('data/2019_income.csv', low_memory=False)
df_income = df_income[df_income["LSOA Code (2011)"].isin(LSOAS_all_of_london)]

df_employment = pd.read_csv('data/2019_employment.csv', low_memory=False)
df_employment = df_employment[df_employment["LSOA Code (2011)"].isin(LSOAS_all_of_london)]

df_stop_and_search = pd.read_csv('data/2019_city_of_london/2019_city_of_london_stop_and_search.csv').append(pd.read_csv('data/2019_metropolitan/2019_metropolitan_stop_and_search.csv'))
df_arrests = df_stop_and_search[df_stop_and_search['Outcome linked to object of search'] == True]

df_searches = df_stop_and_search[df_stop_and_search['Latitude'].notnull()]
df_searches = df_searches[df_searches['Longitude'].notnull()]

df_arrests = df_arrests[df_arrests['Latitude'].notnull()]
df_arrests = df_arrests[df_arrests['Longitude'].notnull()]

In [ ]:
df_metropolitan_street = combine_csvs_in_dir_to_df('multidata/metropolitan/street')
df_metropolitan_street = df_metropolitan_street[df_metropolitan_street["LSOA code"].isin(LSOAS_all_of_london)]

df_cityoflondon_street = combine_csvs_in_dir_to_df('multidata/city-of-london/street')
df_cityoflondon_street = df_cityoflondon_street[df_cityoflondon_street["LSOA code"].isin(LSOAS_all_of_london)]

df_alloflondon_street = df_metropolitan_street.append(df_cityoflondon_street)
df_alloflondon_street = df_alloflondon_street.dropna(axis=0, subset=['Crime type'])

### Group data

In [ ]:
df_alloflondon_street_counts = df_alloflondon_street.groupby('LSOA code').size().to_frame('size')
df_alloflondon_street_counts.reset_index(level=0, inplace=True)
df_alloflondon_street_counts['log(size)'] = df_alloflondon_street_counts['size'].apply(lambda x: math.log(x))
df_alloflondon_street_counts = df_alloflondon_street_counts.rename(columns={"size": "Crime rate", "log(size)": "Crime rate log"})
df_alloflondon_street_counts.head(5)

### Scrape boundary GeoJSON from findthatpostcode.uk 

In [ ]:
if not load_from_file:
    GEOJSON = {}
    LSOA_length = len(LSOAS_all_of_london)
    for idx, LSOA in enumerate(LSOAS_all_of_london):
        res = httpx.get(f'https://findthatpostcode.uk/areas/{LSOA}.geojson')
        body = json.loads(res.content.decode())
        if "message" in body:
            if "boundary not found" in body["message"]:
                print('Boundary not found for LSOA: '+str(LSOA))
                continue
        else:
            print(f"Successfully scraped {LSOA} ({idx}/{LSOA_length})")
        GEOJSON[LSOA] = body
else:
    GEOJSON_FULL = json.load(open('LSOAS_all_of_london.geojson'))

### Merge the different GeoJSON boundaries that were scraped to one object

In [ ]:
if not load_from_file:
    GEOJSON_FULL = {
        "type": "FeatureCollection",
        "features": []
    }

    for GEO in GEOJSON:
        TEMP = GEOJSON[GEO]
        TEMP["features"][0]["id"] = GEO
        GEOJSON_FULL["features"].append(TEMP["features"][0])

    with open('LSOAS_all_of_london.geojson', 'w+') as outfile:
        json.dump(GEOJSON_FULL, outfile)

### Create the Mapbox choropleth map using Plotly

In [ ]:
fig = px.choropleth_mapbox(
    df_income,
    geojson = GEOJSON_FULL,
    locations = 'LSOA Code (2011)',
    color = 'Income Domain Rank (where 1 is most deprived)',
    color_continuous_scale = "Viridis",
    mapbox_style = "carto-positron",
    zoom = 8.6,
    center = {"lat": 51.49, "lon": -0.12},
    opacity = 0.9
)

fig.update_traces(marker_line_width=0)

fig.show()

In [ ]:
fig = px.choropleth_mapbox(
    df_employment,
    geojson = GEOJSON_FULL,
    locations = 'LSOA Code (2011)',
    color = 'Employment Domain Rank (where 1 is most deprived)',
    color_continuous_scale = "Viridis",
    mapbox_style = "carto-positron",
    zoom = 8.6,
    center = {"lat": 51.49, "lon": -0.12},
    opacity = 0.9
)

fig.update_traces(marker_line_width=0)

fig.show()

In [ ]:
fig = go.Figure(go.Scattermapbox(
        lat=list(df_searches['Latitude']),
        lon=list(df_searches['Longitude']),
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=2,
            opacity=1,
            color="orange"
        ),
        text=list(df_searches['Type']),
        name="Searches"
    ))

fig.add_trace(
    go.Scattermapbox(
        lat=list(df_arrests['Latitude']),
        lon=list(df_arrests['Longitude']),
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=5,
            opacity=1,
            color="blue"
        ),
        text=list(df_arrests['Type']),
        name="Arrests"
    )
)

fig.update_layout(
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=51.49,
            lon=-0.12
        ),
        pitch=0,
        zoom=8.6
    )
)

fig.show()

In [ ]:
fig = px.choropleth_mapbox(
    df_alloflondon_street_counts,
    geojson = GEOJSON_FULL,
    locations = 'LSOA code',
    color = 'Crime rate',
    color_continuous_scale = "Viridis",
    mapbox_style = "carto-positron",
    zoom = 8.6,
    center = {"lat": 51.49, "lon": -0.12},
    opacity = 0.9
)

fig.update_traces(marker_line_width=0)

fig.show()

In [ ]:
fig = px.choropleth_mapbox(
    df_alloflondon_street_counts,
    geojson = GEOJSON_FULL,
    locations = 'LSOA code',
    color = 'Crime rate log',
    color_continuous_scale = "Viridis",
    mapbox_style = "carto-positron",
    zoom = 8.6,
    center = {"lat": 51.49, "lon": -0.12},
    opacity = 0.9
)

fig.update_traces(marker_line_width=0)

fig.show()